<a href="https://colab.research.google.com/github/RepnaiaE/DataScience_Emotion-recognition-methods/blob/main/Lab5_Neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Нейронные сети. Нейронная сеть прямого распространения./Neural networks. Neural network of direct distribution.

# Загрузка данных/Loading data

Загружаем данные из ячеек в массивы признаков для каждого набора из предыдущей работы

In [ ]:
import pandas as pd
import numpy as np

ling_features = pd.read_csv(r'C:\Users\voloshina\Documents\Notebooks\Распознавание эмоций\Распознавание эмоций\ru_tweet_corp_ling_features.csv').sample(frac=1)
ling_features_vec = list(ling_features['features'].apply(lambda y: [float(x) for x in y[1:len(y)-1].split(',')]))

In [ ]:
morph_features = pd.read_csv(r'C:\Users\voloshina\Documents\Notebooks\Распознавание эмоций\Распознавание эмоций\ru_tweet_corp_morph_features.csv').sample(frac=1)
morph_features_vec = list(morph_features['coded_pos'].apply(lambda y: [float(x) for x in y[1:len(y)-1].split(',')]))

In [ ]:
tfidf_features = pd.read_csv(r'C:\Users\voloshina\Documents\Notebooks\Распознавание эмоций\Распознавание эмоций\ru_tweet_corp_tfidf.csv').sample(frac=1)
tfidf_features_vec = list(tfidf_features['tfidf'].apply(lambda y: [float(x) for x in y[1:len(y)-1].split(',')]))

In [ ]:
import pandas as pd

concated_features = pd.read_csv(r'C:\Users\voloshina\Documents\Notebooks\Распознавание эмоций\Распознавание эмоций\ru_tweet_corp_tfidf.csv').sample(frac=1)

features_tfidf = list(concated_features['tfidf'].apply(lambda y: [float(x) for x in y[1:len(y)-1].split(',')]))
morph_features_coded_pos = list(concated_features['coded_pos'].apply(lambda y: [float(x) for x in y[1:len(y)-1].split(',')]))
ling_features_features = list(concated_features['features'].apply(lambda y: [float(x) for x in y[1:len(y)-1].split(',')]))

concated_features_vec = []
for tf, morph, ling in zip(features_tfidf, morph_features_coded_pos, ling_features_features):
    concated_features_vec.append(tf + morph + ling)

# Подготовка данных/Data preparation



In [ ]:
!pip install torchvision

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import scipy
import torch.utils.data as data_utils
from sklearn.metrics import classification_report

In [ ]:
from sklearn.model_selection import train_test_split

def getDataLoaders(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
    inputs_train = torch.tensor(X_train)
    targets_train = torch.IntTensor(y_train)

    inputs_test = torch.tensor(X_test)
    targets_test = torch.IntTensor(y_test)

    train = data_utils.TensorDataset(inputs_train, targets_train)
    test = data_utils.TensorDataset(inputs_test, targets_test)

    trainset = torch.utils.data.DataLoader(train, shuffle=True)
    testset = torch.utils.data.DataLoader(test, shuffle=False)

    return trainset, testset, y_test

Метка -1 (негативный) переведена в 0 для дальнейшей классификации:

-1 -> 0 (negative)

 1 (positive)

In [ ]:
ling_train_set, ling_test_set, ling_test_y = getDataLoaders(ling_features_vec,
                                                            [(0 if x == -1 else x) for x in ling_features['ttype'].values])

In [ ]:
morph_train_set, morph_test_set, morph_test_y = getDataLoaders(morph_features_vec,
                                                               [(0 if x == -1 else x) for x in morph_features['ttype'].values])

In [ ]:
tf_idf_train_set, tf_idf_test_set, tf_idf_test_y = getDataLoaders(tfidf_features_vec,
                                                                  [(0 if x == -1 else x) for x in tfidf_features['ttype'].values])

# Модель/Model

In [ ]:
NUM_CLASSES = 2

In [ ]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [ ]:
lingNet = Net(len(ling_features_vec[0]), 32, NUM_CLASSES)
print(lingNet)

Net(
  (fc1): Linear(in_features=6, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)


In [ ]:
morphNet = Net(len(morph_features_vec[0]), 32, NUM_CLASSES)
print(morphNet)

Net(
  (fc1): Linear(in_features=17, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)


In [ ]:
tfIdfNet = Net(len(tfidf_features_vec[0]), 32, NUM_CLASSES)
print(tfIdfNet)

Net(
  (fc1): Linear(in_features=1000, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)


# Обучение/Training

In [ ]:
import torch.optim as optim

def train(net, trainset, testset, y_test, epochs=3):
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    # train
    for epoch in range(epochs):
        for data in trainset:
            X, y = data

            net.zero_grad()
            output = net(X.float())
            loss = F.nll_loss(output, y.long())
            loss.backward()
            optimizer.step()

        print(f"train loss {loss} for epoch {epoch}")

    # test
    ams = []
    with torch.no_grad():
        for data in testset:
            X, y = data

            output = net(X.float())
            for idx, i in enumerate(output):
                ams.append(torch.argmax(i).item())

    print(len(ams))
    print(classification_report(y_test, ams))

In [ ]:
train(lingNet, ling_train_set, ling_test_set, ling_test_y, epochs=10)

train loss 0.6742783188819885 for epoch 0
train loss 0.6783490777015686 for epoch 1
train loss 0.6560860276222229 for epoch 2
train loss 0.734861433506012 for epoch 3
train loss 0.6706293821334839 for epoch 4
train loss 0.6847732067108154 for epoch 5
train loss 0.45956870913505554 for epoch 6
train loss 0.9584934711456299 for epoch 7
train loss 0.6887816190719604 for epoch 8
train loss 0.7434947490692139 for epoch 9
400
              precision    recall  f1-score   support

           0       0.64      0.30      0.41       208
           1       0.52      0.82      0.63       192

    accuracy                           0.55       400
   macro avg       0.58      0.56      0.52       400
weighted avg       0.58      0.55      0.52       400



In [ ]:
train(morphNet, morph_train_set, morph_test_set, morph_test_y, epochs=10)

train loss 0.6090214252471924 for epoch 0
train loss 0.6474200487136841 for epoch 1
train loss 0.9084160327911377 for epoch 2
train loss 0.41185396909713745 for epoch 3
train loss 0.5989606380462646 for epoch 4
train loss 0.8005595207214355 for epoch 5
train loss 0.9279309511184692 for epoch 6
train loss 1.2301185131072998 for epoch 7
train loss 0.6267765164375305 for epoch 8
train loss 0.8288030624389648 for epoch 9
400
              precision    recall  f1-score   support

           0       0.55      0.74      0.63       191
           1       0.66      0.45      0.54       209

    accuracy                           0.59       400
   macro avg       0.60      0.60      0.58       400
weighted avg       0.61      0.59      0.58       400



In [ ]:
train(tfIdfNet, tf_idf_train_set, tf_idf_test_set, tf_idf_test_y, epochs=10)

train loss 0.5595694780349731 for epoch 0
train loss 0.0908379927277565 for epoch 1
train loss 0.7321755886077881 for epoch 2
train loss 0.02971639670431614 for epoch 3
train loss 0.0012288884026929736 for epoch 4
train loss 0.5677809119224548 for epoch 5
train loss 0.0002944036095868796 for epoch 6
train loss 0.23128077387809753 for epoch 7
train loss 5.960446742392378e-06 for epoch 8
train loss 0.005863370839506388 for epoch 9
400
              precision    recall  f1-score   support

           0       0.64      0.60      0.62       204
           1       0.61      0.64      0.63       196

    accuracy                           0.62       400
   macro avg       0.62      0.62      0.62       400
weighted avg       0.62      0.62      0.62       400



# Выводы/Conclusion

Для каждого набора признаков была обучена нейронная сеть со следующими параметрками:
* 3 линейных слоя
* размер скрытого слоя - 32
* скорость обучения - 0.001
* 10 эпох обучения
* оптимизатор Adam
* функция потерь - nll_loss

## Результаты обучения

| Features | F1 score |
|----------|----------|
| Лингвистические | 0.55 |
| Морфологические | 0.59 |
| TF-IDF | 0.62 |

Как итог, при одинаковой конфигурации нейронной сети, лучшие результаты были получены для набора признаков TF-IDF

For each set of features, a neural network was trained with the following parameters:
* 3 line layers
* hidden layer size - 32
* learning rate - 0.001
* 10 learning epochs
* Adam optimizer
* loss function - nll_loss

As a result, with the same neural network configuration, the best results were obtained for the TF-IDF feature set